In [2]:
import pandas as pd

In [32]:
def get_best_model_single(file_path_prefix, k_range):
    inner_scores = []
    params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
    
    for k in range(k_range): #Change to 5 if you are using the binary
        inner_df = pd.read_csv(f'{file_path_prefix}{k}.csv')
        inner_df = inner_df.sort_values('F1',ascending=False).reset_index(drop=True)
        inner_scores += inner_df.to_dict('records')
    
    inner_scores = pd.DataFrame.from_dict(inner_scores).fillna(-1)
    best_model = (inner_scores
                  .groupby(params).agg({
                      'F1':'mean',
                      'ROC AUC':'mean',
                      'precision':'mean',
                      'recall':'mean',
                      'accuracy':'mean'
                      }).sort_values('F1',ascending=False).reset_index()).to_dict('records')[0]
    best_model = pd.DataFrame(best_model,index=[0])
    del inner_scores, inner_df
    return best_model

In [33]:
test2=get_best_model_single(f'/home/dveytia/ORO-map-relevance/outputs/model_selection/climate_mitigation_model_selection_', 3)
test2

,batch_size,weight_decay,learning_rate,num_epochs,class_weight,F1,ROC AUC,precision,recall,accuracy
0,16,0.0,0.00001,2,-1,0.802477,0.938158,0.726099,0.897321,0.879555


In [11]:
# This is just I I wanted to create different lines for the labels, but I think for simplicity I'll just use the macro 
file_path_prefix = f'/home/dveytia/ORO-map-relevance/outputs/model_selection/adapt_to_threat_model_selection_'
inner_scores = []
params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
k=0
inner_df = pd.read_csv(f'{file_path_prefix}{k}.csv')
targets = [x for x in inner_df.columns if 'F1' in x] 
#[x for x in ['micro', 'weighted', 'samples'] for x in targets)
targets.remove('F1 micro')
targets.remove('F1 weighted')
targets.remove('F1 samples')
targets


['F1 macro',
 'F1 - adapt_to_threat.Human',
 'F1 - adapt_to_threat.Natural',
 'F1 - adapt_to_threat.Both']

In [31]:
file_path_prefix = f'/home/dveytia/ORO-map-relevance/outputs/model_selection/adapt_to_threat_model_selection_'
inner_scores = []
params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
k=0

inner_df = inner_df.sort_values('F1 macro',ascending=False).reset_index(drop=True)
inner_scores += inner_df.to_dict('records')
    
inner_scores = pd.DataFrame.from_dict(inner_scores).fillna(-1)

if 'accuracy macro' not in list(inner_scores.columns): # if there is no accuracy macro column set to dummy value
        inner_scores['accuracy macro'] = -999 

best_model = (inner_scores
                  .groupby(params).agg({
                      'F1 macro':'mean',
                      'ROC AUC macro':'mean',
                      'precision macro':'mean',
                      'recall macro':'mean',
                      'accuracy macro':'mean'
                      }).sort_values('F1 macro',ascending=False).reset_index()).to_dict('records')[0]
    
best_model = pd.DataFrame(best_model,index=[0])
best_model.rename(columns={'F1 macro': 'F1', 'ROC AUC macro': 'ROC AUC', 'precision macro':'precision', 'recall macro':'recall', 'accuracy macro':'accuracy'}, inplace=True)

best_model

KeyError: 'F1'

In [19]:
pd.DataFrame(best_model,index=[0])

,batch_size,weight_decay,learning_rate,num_epochs,class_weight,F1 macro,ROC AUC macro,precision macro,recall macro,accuracy macro
0,32,0.0,0.00005,4,"{0: 2.116788321167883, 1: 12.774193548387096, ...",0.42117,0.769914,0.499038,0.499671,-999.0


In [34]:
def get_best_model_multi(file_path_prefix, k_range):
    inner_scores = []
    params = ['batch_size','weight_decay','learning_rate','num_epochs','class_weight']
    
    for k in range(k_range): #Change to 5 if you are using the binary
        inner_df = pd.read_csv(f'{file_path_prefix}{k}.csv')
        inner_df = inner_df.sort_values('F1 macro', ascending=False).reset_index(drop=True)
        inner_scores += inner_df.to_dict('records')
    
    inner_scores = pd.DataFrame.from_dict(inner_scores).fillna(-1)
    
    if 'accuracy macro' not in list(inner_scores.columns): # if there is no accuracy macro column set to dummy value
        inner_scores['accuracy macro'] = -999 
        
    best_model = (inner_scores
                  .groupby(params).agg({
                      'F1 macro':'mean',
                      'ROC AUC macro':'mean',
                      'precision macro':'mean',
                      'recall macro':'mean',
                      'accuracy macro':'mean'
                      }).sort_values('F1 macro',ascending=False).reset_index()).to_dict('records')[0]
    
    best_model = pd.DataFrame(best_model,index=[0])
    best_model.rename(columns={'F1 macro': 'F1', 'ROC AUC macro': 'ROC AUC', 'precision macro':'precision', 'recall macro':'recall', 'accuracy macro':'accuracy'}, inplace=True)
    
    del inner_scores, inner_df
    return best_model

In [36]:
test1=get_best_model_multi(f'/home/dveytia/ORO-map-relevance/outputs/model_selection/adapt_to_threat_model_selection_', 3)
test1

,batch_size,weight_decay,learning_rate,num_epochs,class_weight,F1,ROC AUC,precision,recall,accuracy
0,32,0.0,0.00005,4,"{0: 2.116788321167883, 1: 12.774193548387096, ...",0.470982,0.835704,0.540875,0.494483,-999.0


In [42]:
best_models_all=pd.concat([test1, test2])
best_models_all

,batch_size,weight_decay,learning_rate,num_epochs,class_weight,F1,ROC AUC,precision,recall,accuracy
0,32,0.0,0.00005,4,"{0: 2.116788321167883, 1: 12.774193548387096, ...",0.470982,0.835704,0.540875,0.494483,-999.000000
0,16,0.0,0.00001,2,-1,0.802477,0.938158,0.726099,0.897321,0.879555


In [39]:
test2.shape

(1, 10)